#### Import required packages in the notebook

In [ ]:
#!pip3 install psycopg2-binary

In [1]:
import psycopg2

#### Creating the required postgresql tables and inserting data into the reference tables

In [2]:
print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(
    host="localhost",
    port='5432',
    dbname="postgres_db",
    user="postgres",
    password="123")


cur = conn.cursor()

Connecting to the PostgreSQL database...


#### Creating the required tables along with the relevant constraints

In [3]:
createCmd_raw = """ 
                  CREATE TABLE IF NOT EXISTS restaurant_inspection_raw (
                        camis integer NOT NULL,
                        dba VARCHAR(1000),
                        boro VARCHAR(1000),
                        building VARCHAR(1000),
                        street VARCHAR(1000),
                        zipcode integer,
                        phone integer,
                        cuisine_description VARCHAR(1000),
                        inspection_date TIMESTAMP,
                        action VARCHAR(1000),
                        violation_code VARCHAR(1000),
                        violation_description VARCHAR(1000),
                        critical_flag VARCHAR(1000),
                        score INTEGER,
                        grade VARCHAR(5),
                        grade_date timestamp,
                        record_date timestamp NOT NULL,
                        inspection_type VARCHAR(1000),
                        latitude numeric(18,14),
                        longitude numeric(18,14),
                        community_board VARCHAR(1000),
                        council_district VARCHAR(1000),
                        census_tract VARCHAR(1000),
                        bin VARCHAR(1000),
                        bbl VARCHAR(1000),
                        nta VARCHAR(1000)	
                );
                  """
    
cur.execute(createCmd_raw)

createCmd_boro = """ 
                    CREATE TABLE IF NOT EXISTS borough (
                            boro integer PRIMARY KEY,
                            name VARCHAR(1000)
                    );
                """
    
cur.execute(createCmd_boro)
    
createCmd_restaurants = """ 
                        CREATE TABLE IF NOT EXISTS restaurants (
                                camis integer PRIMARY KEY,
                                dba VARCHAR(1000),
                                boro integer not null,
                                building VARCHAR(1000),
                                street VARCHAR(1000),
                                zipcode integer,
                                phone integer,
                                cuisine_description VARCHAR(1000),
                                latitude numeric(18,14),
                                longitude numeric(18,14)
                        );

                    """
    
cur.execute(createCmd_restaurants)

createCmd_grade = """ 
                  CREATE TABLE IF NOT EXISTS grade (
                            grade  VARCHAR(5) PRIMARY KEY,
                            description VARCHAR(1000)
                  );
                """
    
cur.execute(createCmd_grade)

createCmd_inspection_data = """ 
                  CREATE TABLE IF NOT EXISTS inspection_data (
                            camis integer NOT NULL,
                            inspection_date TIMESTAMP NOT NULL,
                            inspection_type VARCHAR(1000),
                            action VARCHAR(1000),
                            violation_code VARCHAR(1000),
                            violation_description VARCHAR(1000),
                            critical_flag VARCHAR(1000),
                            score INTEGER,
                            grade VARCHAR(5),
                            grade_date timestamp,
                            community_board VARCHAR(1000),
                            council_district VARCHAR(1000),
                            census_tract VARCHAR(1000),
                            bin VARCHAR(1000),
                            bbl VARCHAR(1000),
                            nta VARCHAR(1000),
                            record_date timestamp NOT NULL,
                            CONSTRAINT insp_unique UNIQUE (camis, inspection_date,critical_flag,violation_code,record_date),
                            CONSTRAINT fk_camis FOREIGN KEY(camis) REFERENCES restaurants(camis)

                    );
                """
    
cur.execute(createCmd_inspection_data)
    

conn.commit()


#### Inserting Data into relevant reference tables

In [4]:
# Inserting new information into the borough table

cur.execute("""
              INSERT INTO borough (boro, name)
                VALUES 
                  (1, 'MANHATTAN'),
                  (2, 'BRONX'),
                  (3, 'BROOKLYN'),
                  (4, 'QUEENS'),
                  (5, 'STATEN ISLAND');
            """
          )

conn.commit()

# Inserting new information into the grade table

cur.execute("""
              INSERT INTO grade (grade, description)
                VALUES 
                  ('A', 'Grade A'),
                  ('B', 'Grade B'),
                  ('C', 'Grade C'),
                  ('Z', 'Grade Pending'),
                  ('P', 'Grade Pending issued on re-opening following an initial inspection that resulted in a closure'),
                  ('N', 'Not Yet Graded');

            """
          )

conn.commit()

#### Retrieving all results from the reference tables

In [5]:
queryBoro = 'SELECT * from borough'    
cur.execute(queryBoro)    
print("The number of rows in the borough table: ", cur.rowcount)

row = cur.fetchone()
while row is not None:
    print("boro :", row[0], ", name :", row[1])
    row = cur.fetchone()   

queryGrade = 'SELECT * from grade'    
cur.execute(queryGrade)    
print("\n\nThe number of rows in the grade table: ", cur.rowcount)

row = cur.fetchone()
while row is not None:
    print("grade :", row[0], ", description :", row[1])
    row = cur.fetchone()   



The number of rows in the borough table:  5
boro : 1 , name : MANHATTAN
boro : 2 , name : BRONX
boro : 3 , name : BROOKLYN
boro : 4 , name : QUEENS
boro : 5 , name : STATEN ISLAND


The number of rows in the grade table:  6
grade : A , description : Grade A
grade : B , description : Grade B
grade : C , description : Grade C
grade : Z , description : Grade Pending
grade : P , description : Grade Pending issued on re-opening following an initial inspection that resulted in a closure
grade : N , description : Not Yet Graded


In [6]:
conn.close()